In [1]:
import pandas as pd
import re

In [2]:
path_report_oi = './reports/oferta_individual.xlsx'
path_portafolio = './reports/portafolio.xlsx'
path_past_portafolio = './reports/past_portafolio.xlsx'

In [3]:
#Lista de campos que se usarán del reporte estadístico de Hércules.
fields_report = [
    'Evento',
    'Documento',
    'Tipo Documento',
    'Sedes',
    'Área',
    'Ciclos',
    'Categoría Deportiva',
    'Nivel',
    'Género',
    'Categoría Precio',
    'Categoría Afiliación',
    'creada_por',
    'Estado Cotización',
    'Canal de Pago',
    'Canal de Cotización',
    'Forma de Pago',
    'Franquicias',
    'Precio Base',
    'Costo',
    'Tarifa Plena',
    'Subsidio a la Demanda',
    'Identificador del Cajero',
    'Identificador Máquina',
    'Fecha Registro',
    'Documento Titular',
    'Fecha Cotización',
    'Id Material',
    'Fecha Inicio del Servicio',
    'Fecha Fin del Servicio',
    'Id Servicio',
    'Edad',
    'Fecha de Nacimiento',
    'Valor',
    'Motivo de Cambio',
]

In [117]:
# Importando la data desde el reporte.
dataset=pd.read_excel(path_report_oi, usecols=fields_report)

In [118]:
# Se eliminan todos los registros donde no haya un valor en la sede o en la fecha de reserva.
dash_dataset = dataset.dropna(subset=['Sedes', 'Fecha Inicio del Servicio'])
print(
    f'Se han eliminado {dataset['Evento'].count()-dash_dataset['Evento'].count()} filas que no tienen registros para las sedes o en la fecha de reserva.'
)

# Inserción de columna 'Tipo de Oferta' al final del dataset, asignádole un valor ''.
dash_dataset.insert(loc=len(dash_dataset.columns), column='Tipo de Oferta', value='')

# Eliminación del prefijo '-F' en los código SAP (campo 'Id Material') de la oferta FOSFEC.
dash_dataset['Id Material'].replace(r'(\d{8})-F', r'\g<1>', inplace=True, regex=True)
dash_dataset['Id Material'] = pd.to_numeric(dash_dataset['Id Material'])

Se han eliminado 112 filas que no tienen registros para las sedes o en la fecha de reserva.


/tmp/ipykernel_12847/1102828524.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dash_dataset["Id Material"].replace(r"(\d{8})-F", r"\g<1>", inplace=True, regex=True)
/tmp/ipykernel_12847/1102828524.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dash_dataset["Id Material"] = pd.to_numeric(dash_dataset["Id Material"])


In [119]:
# Patrones empleados en el campo 'Motivo de cambio' usando RegEx y con ello asignar las ofertas por Excedentes del 55% y cobro a empresas por una venta individual.
pat_exc55 = r'exc\w*\s*\d*[%]*[^excel]'
pat_empresarial = r'empr\w*\s*|conv\w*\s*|nit\s*\d*|contr|se factura'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51039 entries, 0 to 51150
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Evento                     51039 non-null  object 
 1   Documento                  51039 non-null  int64  
 2   Tipo Documento             51039 non-null  object 
 3   Sedes                      51039 non-null  object 
 4   Área                       51039 non-null  object 
 5   Ciclos                     51039 non-null  object 
 6   Categoría Deportiva        51039 non-null  object 
 7   Nivel                      51039 non-null  object 
 8   Género                     51039 non-null  object 
 9   Categoría Precio           51039 non-null  object 
 10  Categoría Afiliación       51039 non-null  object 
 11  creada_por                 51039 non-null  object 
 12  Estado Cotización          51039 non-null  object 
 13  Canal de Pago              27308 non-null  obj

In [120]:
# Asignación tipo de oferta 'Pago con novedad'.
dash_dataset.loc[dash_dataset['Canal de Pago'].isna(), 'Tipo de Oferta']='Pago con Novedad'

# Asignación tipo de oferta 'FOSFEC'.
dash_dataset.loc[dash_dataset['Evento']=='fosfec', 'Tipo de Oferta'] = 'FOSFEC'

# Asignación tipo de oferta 'Pago empresarial'.
dash_dataset.loc[dash_dataset['Motivo de Cambio'].notna() & dash_dataset['Motivo de Cambio'].str.contains(pat_empresarial, case=False, regex=True), 'Tipo de Oferta'] = 'Pago Empresarial'

# Asignación tipo de oferta 'Excedentes del 55%'.
dash_dataset.loc[(dash_dataset['Ciclos'].str.contains(pat_exc55, case=False, regex=True)) | dash_dataset['Motivo de Cambio'].str.contains(pat_exc55, case=False, regex=True), 'Tipo de Oferta'] = 'Excedentes del 55%'

# Asignación tipo de oferta 'Venta Directa'.
dash_dataset.loc[(dash_dataset['Canal de Pago'].notna()) & (dash_dataset['Motivo de Cambio'].isna()), 'Tipo de Oferta'] = 'Venta Directa'

In [121]:
# Lista de campos del portafolio que se usarán para cruzar con el reporte estadístico.
fields_portafolio = [
    'UNIDAD DE NEGOCIO',
    'LINEA',
    'CODIGO SAP',
    'MATERIAL',
    'NUM. PARTICIPANTES MAX',
]

In [108]:
# última versión del portafolio.
portafolio = pd.read_excel(path_portafolio, usecols=fields_portafolio)

# Archivo que contiene materiales pasados que no están en la última actualización del portafolio.
past_portafolio = pd.read_excel(path_past_portafolio, usecols=fields_portafolio)

full_portafolio=pd.concat([portafolio, past_portafolio])


# Selecciona del portafolio sólo las filas donde la unidad de negocio coincida con las indicacadas en la lista 'UNIDADES DE NEGOCIO', pues son las unidades que ofertan servicios en Hércules.
UNIDADES_NEGOCIO = ['EDUCACIÓN', 'CULTURA', 'ESPARCIMIENTO', 'DESARROLLO SOCIAL']
portafolio_hercules = full_portafolio[
     full_portafolio['UNIDAD DE NEGOCIO'].isin(UNIDADES_NEGOCIO)
]

# Elimina las fila donde se encuentre el repetido el código SAP.
portafolio_hercules.drop_duplicates('CODIGO SAP', inplace=True, keep='first')

/tmp/ipykernel_12847/2821458284.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portafolio_hercules.drop_duplicates("CODIGO SAP", inplace=True, keep="first")


In [110]:
portafolio_hercules.to_csv('full.csv', index=False, index_label=False)

In [122]:
# Se ejecuta un left-join trayendo los campos de portafolio cruzados a través del Código SAP.
final_dataset = pd.merge(
    dash_dataset,
    portafolio_hercules[
        [
            'UNIDAD DE NEGOCIO',
            'LINEA',
            'CODIGO SAP',
            'NUM. PARTICIPANTES MAX',
        ]
    ],
    left_on='Id Material',
    right_on='CODIGO SAP',
    how='left',
    # validate='1:m'
    
)

In [123]:
final_dataset.to_csv('dash_dataset.csv', index = False, index_label=False)

In [29]:
df=pd.read_csv('./assets/data/dash_dataset.csv', header=0)


In [33]:
df["Documento"].shape[0]

51039